In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp
import networkx as nx

#visualizing results
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
%matplotlib inline
import seaborn as sns
sns.set_context('poster')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn import metrics
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans

import scipy.cluster.hierarchy as shc

### code borrowed heavily from: https://towardsdatascience.com/visualising-stocks-correlations-with-networkx-88f2ee25362e

### Get data and create corr matrix

In [ ]:
data_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/final_data/data_final_earliest.csv'

In [ ]:
#read in csv containing data from all surveys and all visitseqs
data = pd.read_csv(data_path, index_col=0)
data = pd.DataFrame(data = data)

print('Original data shape:\n', data.shape, '\n')
print(data.info())
data.head(1)

In [ ]:
data['Group_num'] = [0.0 if x == 'C' else 1.0 for x in data['Group']]

In [ ]:
data[data['DTI_FA_post_corona_radiata_R'] > 0].groupby('VisitSeq')['Group_num'].value_counts()

In [ ]:
data['DTI_FA_corticospinal'] = data[['DTI_FA_corticospinal_R', 'DTI_FA_corticospinal_L']].mean(axis=1)
data['DTI_FA_med_lemniscus'] = data[['DTI_FA_med_lemniscus_R', 'DTI_FA_med_lemniscus_L']].mean(axis=1)
data['DTI_FA_inf_cerebll_pedunc'] = data[['DTI_FA_inf_cerebll_pedunc_R', 'DTI_FA_inf_cerebll_pedunc_L']].mean(axis=1)
data['DTI_FA_sup_cerebll_pedunc'] = data[['DTI_FA_sup_cerebll_pedunc_R', 'DTI_FA_sup_cerebll_pedunc_L']].mean(axis=1)
data['DTI_FA_cerebr_pedunc'] = data[['DTI_FA_cerebr_pedunc_R', 'DTI_FA_cerebr_pedunc_L']].mean(axis=1)
data['DTI_FA_ant_int_capsule'] = data[['DTI_FA_ant_int_capsule_R', 'DTI_FA_ant_int_capsule_L']].mean(axis=1)
data['DTI_FA_post_int_capsule'] = data[['DTI_FA_post_int_capsule_R', 'DTI_FA_post_int_capsule_L']].mean(axis=1)
data['DTI_FA_retrolenticular_int_capsule'] = data[['DTI_FA_retrolenticular_int_capsule_R', 'DTI_FA_retrolenticular_int_capsule_L']].mean(axis=1)
data['DTI_FA_ant_corona_radiata'] = data[['DTI_FA_ant_corona_radiata_R', 'DTI_FA_ant_corona_radiata_L']].mean(axis=1)
data['DTI_FA_sup_corona_radiata'] = data[['DTI_FA_sup_corona_radiata_R', 'DTI_FA_sup_corona_radiata_L']].mean(axis=1)
data['DTI_FA_post_corona_radiata_R'] = data[['DTI_FA_post_corona_radiata_R', 'DTI_FA_post_corona_radiata_L']].mean(axis=1)
data['DTI_FA_post_thal_rad'] = data[['DTI_FA_post_thal_rad_R', 'DTI_FA_post_thal_rad_L']].mean(axis=1)
data['DTI_FA_sag_stratum'] = data[['DTI_FA_sag_stratum_R', 'DTI_FA_sag_stratum_L']].mean(axis=1)
data['DTI_FA_ext_capsule'] = data[['DTI_FA_ext_capsule_R', 'DTI_FA_ext_capsule_L']].mean(axis=1)
data['DTI_FA_cingulum_cg'] = data[['DTI_FA_cingulum_cg_R', 'DTI_FA_cingulum_cg_L']].mean(axis=1)
data['DTI_FA_cingulum_hipp'] = data[['DTI_FA_cingulum_hipp_R', 'DTI_FA_cingulum_hipp_L']].mean(axis=1)
data['DTI_FA_fornix_striaterm'] = data[['DTI_FA_fornix_striaterm_R', 'DTI_FA_fornix_striaterm_L']].mean(axis=1)
data['DTI_FA_sup_long_fascic'] = data[['DTI_FA_sup_long_fascic_R', 'DTI_FA_sup_long_fascic_L']].mean(axis=1)
data['DTI_FA_sup_frontocc_fascic'] = data[['DTI_FA_sup_frontocc_fascic_R', 'DTI_FA_sup_frontocc_fascic_L']].mean(axis=1)
data['DTI_FA_unc_fascic'] = data[['DTI_FA_unc_fascic_R', 'DTI_FA_unc_fascic_L']].mean(axis=1)
data['DTI_FA_tapetum'] = data[['DTI_FA_tapetum_R', 'DTI_FA_tapetum_L']].mean(axis=1)

In [ ]:
data_first = data[(data['VisitSeq'] == 1) & (data['DTI_FA_post_corona_radiata_R'] > 0)]

data_first_C = data_first[data_first['Group_num'] == 0]
data_first_T = data_first[data_first['Group_num'] == 1]

DTI_FA = ['DTI_FA_mid_cerebll_pedunc', 'DTI_FA_pontine_xing_tract',
       'DTI_FA_genu_cc', 'DTI_FA_body_cc', 'DTI_FA_splenium_cc',
       'DTI_FA_fornix', 'DTI_FA_corticospinal',
       'DTI_FA_med_lemniscus', 'DTI_FA_inf_cerebll_pedunc',
       'DTI_FA_sup_cerebll_pedunc', 'DTI_FA_cerebr_pedunc',
       'DTI_FA_ant_int_capsule', 'DTI_FA_post_int_capsule',
       'DTI_FA_retrolenticular_int_capsule', 'DTI_FA_ant_corona_radiata',
       'DTI_FA_sup_corona_radiata', 'DTI_FA_post_thal_rad',
       'DTI_FA_sag_stratum', 'DTI_FA_ext_capsule', 'DTI_FA_cingulum_cg',
       'DTI_FA_cingulum_hipp', 'DTI_FA_fornix_striaterm',
       'DTI_FA_sup_long_fascic', 'DTI_FA_sup_frontocc_fascic',
       'DTI_FA_unc_fascic', 'DTI_FA_tapetum']

data_first_C_DTI = data_first_C[DTI_FA].corr()
data_first_T_DTI = data_first_T[DTI_FA].corr()

regions = data_first_C_DTI.index.values

data_first_C_DTI = np.asmatrix(data_first_C_DTI)
data_first_T_DTI = np.asmatrix(data_first_T_DTI)

### Create and viz network

In [ ]:
#function to create and display networks from the correlatin matrix. 

def create_corr_network_1(G, corr_direction):
    #crates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    #Creates a copy of the graph
    H = G.copy()

    ##Checks all the edges and removes some based on corr_direction
    edge1, edge2 = zip(*edges)
    for e1, e2, w in zip(edge1, edge2, weights):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0
        if corr_direction == "positive":
            if w<.5:
                H.remove_edge(e1, e2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        if corr_direction == "negative":
            if w>=0:
                H.remove_edge(e1, e2)
                
    #positions
    positions=nx.circular_layout(H)
    edges,weights = zip(*nx.get_edge_attributes(H,'weight').items())
    
    #####calculates the degree of each node
    d = nx.degree(H)
    #####creates list of nodes and a list their degrees that will be used later for their sizes
    nodelist, node_sizes = zip(*d)
    
    #Figure size
    plt.figure(figsize=(15,15))

    #draws nodes
    nx.draw_networkx_nodes(H,positions,node_color='#DA70D6',nodelist=nodelist,
                           #####the node size will be now based on its degree
                           node_size=tuple([x**3 for x in node_sizes]),alpha=0.8)
    
    #Styling for labels
    nx.draw_networkx_labels(H, positions, font_size=8, 
                            font_family='sans-serif')
    
    ### increases the value of weights, so that they are more visible in the graph
    weights = tuple([(1+abs(x))**2 for x in weights])

    ###edge colors based on weight direction
    if corr_direction == "positive":
        edge_colour = plt.cm.GnBu 
    else:
        edge_colour = plt.cm.PuRd

    #draws the edges
    nx.draw_networkx_edges(H, positions, edgelist=edges,style='solid',
                      ###adds width=weights and edge_color = weights 
                      ###so that edges are based on the weight parameter 
                      ###edge_cmap is for the color scale based on the weight
                      ### edge_vmin and edge_vmax assign the min and max weights for the width
                      width=weights, edge_color = weights, edge_cmap = edge_colour,
                      edge_vmin = min(weights), edge_vmax=max(weights))

    # displays the graph without axis
    plt.axis('off')
    #saves image
    plt.savefig("part1.png", format="PNG")
    plt.show() 

#### Sham

In [ ]:
#Crates graph using the data of the correlation matrix
G = nx.from_numpy_matrix(data_first_C_DTI)

#relabels the nodes to match the  stocks names
G = nx.relabel_nodes(G,lambda x: regions[x])

In [ ]:
print(nx.to_connected(G))
create_corr_network_1(G, 'negative')

In [ ]:
create_corr_network_1(G, 'positive')

#### Blast

In [ ]:
#Crates graph using the data of the correlation matrix
G = nx.from_numpy_matrix(data_first_T_DTI)

#relabels the nodes to match the  stocks names
G = nx.relabel_nodes(G,lambda x: regions[x])

In [ ]:
create_corr_network_1(G, 'negative')

In [ ]:
create_corr_network_1(G, 'positive')